# Haal maxima van belastingen op

#  importeer de benodigde functies

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

from toolbox_continu_inzicht import Config, DataAdapter
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_rws_config.yaml")
c.lees_config()
data = DataAdapter(config=c)

RWS_webservice = LoadsWaterwebservicesRWS(data_adapter=data)

lst_data = RWS_webservice.run(input="BelastingLocaties", output="Waterstanden")

Configureer als volgt, zonder getij
```yaml
GlobalVariables:
    rootdir: "data_sets"  
    moments: [-24,0,24,48] 

    LoadsMaxima:
        tide: False

DataAdapter:
    waterstanden: 
        type: csv
        path: "hidden_waterstanden.csv"
        sep: ";"
    maxima:   
        type: csv
        path: "hidden_waterstanden_maxima.csv"       
```

In [ ]:
from toolbox_continu_inzicht.loads import LoadsToMoments

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_to_moments_config.yaml")
c.lees_config()
data = DataAdapter(config=c)

maxima = LoadsToMoments(data_adapter=data)

maxima.run(input="waterstanden", output="maxima")

In [ ]:
df_out = RWS_webservice.df_out

In [ ]:
# split de dataframe in tweeën & process
values_nan = df_out[df_out["value"] == -999].index
for val in values_nan:
    df_out.loc[val, "value"] = np.nan
df_out.sort_index()
df_plot = df_out.set_index(df_out["date_time"], drop=True)
df_plot = df_plot[df_plot["measurement_location_id"] == 8420]
measurement_index = df_plot["value_type"] == "verwachting"
df_plot_measurements = df_plot[measurement_index][["value"]]
df_plot_forecast = df_plot[~measurement_index][["value"]]

In [ ]:
df_plot_moments = maxima.df_out

In [ ]:
df_plot_moments

In [ ]:
df_plot_moments[df_plot_moments["measurement_location_id"] == 8420]

In [ ]:
from datetime import timedelta

In [ ]:
fig, ax = plt.subplots()
df_plot_measurements.plot(color="C0", ax=ax)
df_plot_forecast.plot(color="C1", ax=ax)
for val in df_plot_moments[df_plot_moments["measurement_location_id"] == 8420].index:
    ax.axvline(val, color="C2", label="moment")

time_series_moments = df_plot_moments[
    df_plot_moments["measurement_location_id"] == 8420
].index
# maak de grafiek iets breder om de lijnen te zien.
ax.set_xlim(
    (
        min(time_series_moments) - timedelta(hours=1),
        max(time_series_moments) + timedelta(hours=1),
    )
)
ax.legend(["Prediction", "Measurement", "moment"])

### Herhaal met getij

Configureer als volgt, met getij
```yaml
GlobalVariables:
    rootdir: "data_sets"  
    moments: [-24,0,24,48] 

    LoadsMaxima:
        tide: True

DataAdapter:
    waterstanden: 
        type: csv
        path: "hidden_waterstanden.csv"
        sep: ";"
    maxima:   
        type: csv
        path: "hidden_waterstanden_maxima.csv"       
```

In [ ]:
from toolbox_continu_inzicht.loads import LoadsMatroos

test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_noos_matroos_fcst_config.yaml")
c.lees_config()
data = DataAdapter(config=c)

matroos_fsct = LoadsMatroos(data_adapter=data)
matroos_fsct.run(input="BelastingLocaties", output="Waterstanden")

In [ ]:
df_plot_getij = matroos_fsct.df_out
df_plot_getij

In [ ]:
df_plot_getij.measurement_location_id.unique()

In [ ]:
test_data_sets_path = Path.cwd() / "data_sets"
c = Config(config_path=test_data_sets_path / "loads_to_moments_tide_config.yaml")
c.lees_config()
data = DataAdapter(config=c)

maxima_getij = LoadsToMoments(data_adapter=data)

maxima_getij.run(input="waterstanden", output="maxima")

In [ ]:
df_plot_moments_getij = maxima_getij.df_out

In [ ]:
measurement_location_id = 7

In [ ]:
df_plot_getij["measurement_location_id"] = df_plot_getij[
    "measurement_location_id"
].apply(lambda x: int(x))  # TODO fix this issue
df_plot_measurements = df_plot_getij[
    df_plot_getij["measurement_location_id"] == measurement_location_id
]
df_plot_measurements = df_plot_measurements.set_index("date_time")

In [ ]:
from datetime import datetime, timezone

In [ ]:
fig, ax = plt.subplots()
df_plot_measurements["value"].plot(color="C0", ax=ax, label="Verwachting")

for val in df_plot_moments_getij[
    df_plot_moments_getij["measurement_location_id"] == measurement_location_id
].index:
    ax.axvline(val, color="C2", label="moment aangepast door getij")

time_series_getij = df_plot_moments_getij[
    df_plot_moments_getij["measurement_location_id"] == measurement_location_id
].index
# maak de grafiek iets breder om de lijnen te zien.
ax.set_xlim(
    (
        min(time_series_getij) - timedelta(hours=1),
        max(time_series_getij) + timedelta(hours=1),
    )
)
dt_now = datetime.now(timezone.utc)
t_now = datetime(
    dt_now.year,
    dt_now.month,
    dt_now.day,
    dt_now.hour,
    0,
    0,
).replace(tzinfo=timezone.utc)
for moment in c.global_variables["moments"]:
    ax.axvline(t_now + timedelta(hours=moment), color="C3", ls="--", label="Moment")

ax.set_xlim(
    (
        min(time_series_moments) - timedelta(hours=5),
        max(time_series_moments) + timedelta(hours=5),
    )
)

handles, labels = ax.get_legend_handles_labels()
handles = handles[:2] + [handles[6]]
labels = ["Verwachting"] + labels[4:6]
ax.legend(handles=handles, labels=labels, bbox_to_anchor=(1, 1))